# Create Fact Table

**Reading data from silver layer**

In [0]:
df_silver = spark.sql("SELECT * FROM PARQUET.`abfss://silver@tahirbucket.dfs.core.windows.net/carsales`")

In [0]:
df_silver.printSchema()

In [0]:
df_silver.display()

**Reading all the DIMENSION Tables**

In [0]:
df_model  = spark.sql("SELECT * FROM cars_catalog.gold.dim_model")
df_branch = spark.sql("SELECT * FROM cars_catalog.gold.dim_branch")
df_dealer = spark.sql("SELECT * FROM cars_catalog.gold.dim_dealer")
df_date   = spark.sql("SELECT * FROM cars_catalog.gold.dim_date")

**Bringing keys to the FACT Table**

In [0]:
df_fact = df_silver \
    .join(df_model, df_silver['model_id'] == df_model['model_id'], how = 'left') \
    .join(df_branch, df_silver['branch_id'] == df_branch['branch_id'], how = 'left') \
    .join(df_dealer, df_silver['dealer_id'] == df_dealer['dealer_id'], how = 'left') \
    .join(df_date, df_silver['date_id'] == df_date['date_id'], how = 'left') \
    .select(
        df_silver['revenue'],
        df_silver['units_sold'],
        df_silver['rev_per_unit'],
        df_model['dim_model_key'],
        df_branch['dim_branch_key'],
        df_dealer['dim_dealer_key'],
        df_date['dim_date_key']
    )

In [0]:
df_fact.display()

**Writing FACT Table**

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists('cars_catalog.gold.factsales'):
    delta_table = DeltaTable.forName(spark, 'cars_catalog.gold.factsales')
    
    delta_table.alias('target').merge(df_fact.alias('source'), 'target.dim_model_key = source.dim_model_key AND target.dim_branch_key = source.dim_branch_key AND target.dim_dealer_key = source.dim_dealer_key AND target.dim_date_key = source.dim_date_key') \
        .whenMatchedUpdateAll() \
        .whenNotMatchedInsertAll() \
        .execute()    
else:
    df_fact \
        .write.format('delta') \
        .mode('Overwrite') \
        .option('path', 'abfss://gold@tahirbucket.dfs.core.windows.net/factsales') \
        .saveAsTable('cars_catalog.gold.factsales')

In [0]:
%sql
SELECT * FROM cars_catalog.gold.factsales